In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -mkdir /tmp/drivers

In [3]:
!hdfs dfs -ls /tmp/drivers/*

ls: `/tmp/drivers/*': No such file or directory


In [4]:
!hdfs dfs -tail /tmp/drivers/tbl0.csv
!hdfs dfs -tail /tmp/drivers/tbl1.csv

tail: `/tmp/drivers/tbl0.csv': No such file or directory
tail: `/tmp/drivers/tbl1.csv': No such file or directory


In [28]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
--
DROP TABLE IF EXISTS tbl1;
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1; 

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 0.64 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.061 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 1.282 seconds
--
DROP TABLE IF EXISTS tbl1;
OK
Time taken: 0.396 seconds
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.072 seconds
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1; 
Loading data to table default.tbl1
OK
Time taken: 0.804 seconds


In [31]:
%%hive
SELECT * FROM tbl0;
SELECT * FROM tbl1;

SELECT * FROM tbl0;
OK
1	D	4	2016-06-25	["a","e","c","d"]	{"bb":10,"dd":20,"cc":40}
2	C	4	2015-05-14	["a","c"]	{"dd":40,"bb":20,"cc":10}
3	D	6	2014-12-26	["b","d"]	{"aa":10,"bb":40}
4	D	5	2016-06-25	["a","c","e","b","d"]	{"bb":40,"dd":20,"aa":10,"cc":30}
5	C	7	2016-05-23	["d","e","c"]	{"cc":10,"aa":20}
6	A	2	2018-06-14	["a","d"]	{"aa":20}
7	B	3	2014-12-22	["a","e","d"]	{"cc":20}
8	C	6	2015-08-20	["d","a","c","e"]	{"cc":10,"aa":20}
9	B	3	2017-12-08	["b","a","c","e"]	{"cc":40,"dd":10,"aa":30,"bb":20}
10	B	7	2015-07-28	["c","d","e","a","b"]	{"aa":10,"dd":40,"cc":30}
Time taken: 0.22 seconds, Fetched: 10 row(s)
SELECT * FROM tbl1;
OK
1	7	bb	{"A":0,"B":4,"C":1,"D":3,"E":5}
2	13	Cc	{"A":4,"B":1,"C":0,"D":5,"E":2}
3	11	dD	{"A":5,"B":4,"C":3,"D":1,"E":0}
4	3	BB	{"A":2,"B":4,"C":3,"D":5,"E":1}
5	14	cc	{"A":4,"B":0,"C":2,"D":5,"E":3}
6	8	cC	{"A":4,"B":5,"C":1,"D":2,"E":3}
7	2	DD	{"A":0,"B":5,"C":2,"D":4,"E":3}
8	6	dd	{"A":4,"B":2,"C":5,"D":3,"E":0}
9	9	aa	{"A":1,"B":4,"C":2,"D":5,"E":3}
10	4	Bb	

In [39]:
%%hive
SELECT
    t1.c1,
    t1.c2,
    map_get(map(t2.c4,"one"))
FROM
    tbl0
    t1
JOIN (
    SELECT
c1,
c4 
    FROM tbl1 ) t2
ON
    (t1.c1 = t2.c1  );

SELECT
    t1.c1,
    t1.c2,
    map_get(map(t2.c4))
FROM
    tbl0
    t1
JOIN (
    SELECT
c1,
c4 
    FROM tbl1 ) t2
ON
    (t1.c1 = t2.c1  );
FAILED: SemanticException [Error 10011]: Invalid function map_get


In [35]:
%%hive
SELECT  fec, word ,count(*) FROM (
SELECT substr (c4,1,4) AS fec , word
FROM tbl0 lateral view explode(split(c4, '\\:')) temp_table AS word
) x
GROUP BY  fec ,word;

SELECT  fec, word ,count(*) FROM (
SELECT substr (c4,1,4) AS fec , word
FROM tbl0 lateral view explode(split(c4, '\\:')) temp_table AS word
) x
GROUP BY  fec ,word;
Query ID = root_20200208200414_04332bc8-8308-4950-8a97-65c9a1ddbaf5
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1581190519402_0004, Tracking URL = http://aa1fa96f3e93:8088/proxy/application_1581190519402_0004/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1581190519402_0004
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-02-08 20:04:22,632 Stage-1 map = 0%,  reduce = 0%
2020-02-08 20:04:30,484 Stage-1 map = 1

In [26]:
%%hive
SELECT   lateral view explode(c4) AS word
FROM tbl1;

SELECT   lateral view explode(c4) AS word
FROM tbl1;
NoViableAltException(163@[])
	at org.apache.hadoop.hive.ql.parse.HiveParser_SelectClauseParser.selectClause(HiveParser_SelectClauseParser.java:908)
	at org.apache.hadoop.hive.ql.parse.HiveParser.selectClause(HiveParser.java:41988)
	at org.apache.hadoop.hive.ql.parse.HiveParser.atomSelectStatement(HiveParser.java:36720)
	at org.apache.hadoop.hive.ql.parse.HiveParser.selectStatement(HiveParser.java:36987)
	at org.apache.hadoop.hive.ql.parse.HiveParser.regularBody(HiveParser.java:36633)
	at org.apache.hadoop.hive.ql.parse.HiveParser.queryStatementExpressionBody(HiveParser.java:35822)
	at org.apache.hadoop.hive.ql.parse.HiveParser.queryStatementExpression(HiveParser.java:35710)
	at org.apache.hadoop.hive.ql.parse.HiveParser.execStatement(HiveParser.java:2284)
	at org.apache.hadoop.hive.ql.parse.HiveParser.statement(HiveParser.java:1333)
	at org.apache.hadoop.hive.ql.parse.ParseDriver.parse(ParseDriver.java:208)
	at org.apache.hadoop.hive